In [1]:
import numpy as np

In [57]:
# с семинара

num_companies = 4  # кол-во компаний
capital = 100  # в тыс. руб.
step = 10
profit_of_investment = [[0, 7, 13, 20, 33, 35],
                        [0, 8, 12, 21, 28, 40],
                        [0, 10, 11, 19, 30, 34],
                        [0, 6, 9, 24, 30]]

In [58]:
# из моего варианта

num_companies = 4  # кол-во компаний
capital = 700  # в тыс. руб.
step = 100
profit_of_investment = [[0, 28, 45, 65, 78, 90, 102, 113],
                        [0, 25, 41, 55, 65, 75, 80, 85],
                        [0, 15, 25, 40, 50, 62, 73, 82],
                        [0, 20, 33, 42, 48, 53, 56, 58]]
X_matrix = [[i for i in range(0, min(capital + step, step * len(profit_of_investment[j])), step)] for j in
            range(0, len(profit_of_investment))] # массив из массивов индексов возможных вложений (для не квадратной матрицы profit_of_investment)

In [43]:
# итоговая функция

def foo(num_companies, capital, A: np.array):
    """
    :param num_companies: количество предприятий
    :param capital: капитал
    :param profit_of_investment: двумерный массив
    :return:
    """
    F = {}  # F[(k, i)] means F_k(i)
    for k in range(num_companies - 1, -1, -1):  # идем от (num_companies - 1) до 0
        X = [i for i in range(0, capital + step, step)]  # возможные варианты x
        X_indexes = [i for i in range(0, capital + step, step)]  # возможные варианты индексов X
        print("Расчет F(k = {0})".format(k))
        print("При X = {0}".format(X))
        for x in X:
            if X.index(x) < len(A[k]):
                if k == num_companies - 1:
                    F[(k, x)] = A[k][X.index(x)]
                else:
                    U = [i for i in range(0, x + step, step)]  # возможные варианты u
                    print("При U = {0}".format(U))
                    print("max({0})".format([A[k][X.index(x)] + F[(k + 1, x - u)] for u in U]))
                    print("Debug: ", k, X.index(x), A[k][X.index(x)], F[(k + 1, x)])
                    F[(k, x)] = np.max([A[k][u] + F[(k + 1, x - u)] for u in
                                        U])  # создаем массив из возможных F и находим в нем максимум
                print("F[({0}, {1})] = ".format(k, x), F[(k, x)])

    print("Answer = ", F[(0, capital)])

    for i in range(1):
        # print("Variant 1:")
        pass
        for j in range(1):
            # print("It should be spent (*amount1*) on company (*number1*)")
            # print("Total profit is (*target value*))")
            pass

In [44]:
foo(num_companies, capital, profit_of_investment)

Расчет F(k = 3)
При X = [0, 100, 200, 300, 400, 500, 600, 700]
F[(3, 0)] =  0
F[(3, 100)] =  20
F[(3, 200)] =  33
F[(3, 300)] =  42
F[(3, 400)] =  48
F[(3, 500)] =  53
F[(3, 600)] =  56
F[(3, 700)] =  58
Расчет F(k = 2)
При X = [0, 100, 200, 300, 400, 500, 600, 700]
При U = [0]
max([0])
Debug:  2 0 0 0
F[(2, 0)] =  0
При U = [0, 100]
max([35, 15])
Debug:  2 1 15 20


IndexError: index 100 is out of bounds for axis 0 with size 8